In [1]:
import os
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.53


In [2]:
!jupyter nbextension list

Known nbextensions:
  config dir: /data/home/azureuser/.jupyter/nbconfig
    notebook section
      azureml_widgets/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /data/anaconda/envs/py35/etc/jupyter/nbconfig
    notebook section
      nbpresent/js/nbpresent.min disabled
      azureml_widgets/extension  enabled 
      - Validating: OK
      plotlywidget/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
      nb_conda/main  enabled 
      - Validating: OK
    tree section
      nb_conda/tree  enabled 
      - Validating: OK


In [3]:
#!jupyter nbextension install --py --user azureml.widgets

In [4]:
#!jupyter nbextension enable azureml.widgets --user --py

In [5]:
#!pip install azureml.train
#!pip install azureml.widgets


In [6]:
#!pip install --upgrade azureml-sdk[notebooks,explain,automl]

In [7]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

# Compute Environment

In [8]:
cluster = 'Workshop'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)
    

Found existing compute target


# Manage Data

In [9]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = '.', target_path = 'mnist', show_progress = True)

Uploading an estimated of 46 files
Target already exists. Skipping upload for mnist/.gitignore
Target already exists. Skipping upload for mnist/amlcallback.py
Target already exists. Skipping upload for mnist/control.ipynb
Target already exists. Skipping upload for mnist/LICENSE
Target already exists. Skipping upload for mnist/README.md
Target already exists. Skipping upload for mnist/requirements.txt
Target already exists. Skipping upload for mnist/train.py
Target already exists. Skipping upload for mnist/.git/config
Target already exists. Skipping upload for mnist/.git/description
Target already exists. Skipping upload for mnist/.git/HEAD
Target already exists. Skipping upload for mnist/.git/index
Target already exists. Skipping upload for mnist/.git/packed-refs
Target already exists. Skipping upload for mnist/.git/hooks/applypatch-msg.sample
Target already exists. Skipping upload for mnist/.git/hooks/commit-msg.sample
Target already exists. Skipping upload for mnist/.git/hooks/post-u

# Optimize hyperparameters

In [12]:
# same as above but increase the max_steps and remove the parameters
script_params={
    '--data': mnist_data,
    '--epochs': 100
}

mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_requirements_file='requirements.txt')

## hyperparameter search

In [13]:
ps = RandomParameterSampling(
    {
        '--layer': choice(128, 512, 1024),
        '--lr': loguniform(-15, -3),
        '--batch': choice(16, 32, 64, 128)
    }
)

early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 10,
                                            max_concurrent_runs = 5)

hd_run = mnist.submit(hyperdrive_run_config)

RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…